In [137]:
# Welcome to your new notebook
# Type here in the cell editor to add code!


StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 139, Finished, Available, Finished, False)

In [138]:
df = spark.read.format("csv").option("header","true").load("Files/Bronzebronze_sales_orders/sales_orders.csv")
# df now is a Spark DataFrame containing CSV data from "Files/Bronzebronze_sales_orders/sales_orders.csv".
display(df)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 140, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, acfa652d-5ecc-467e-a248-fac0baad6d75)

In [139]:
df.printSchema()

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 141, Finished, Available, Finished, False)

root
 |-- SalesID: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- PlantID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- UnitsSold: string (nullable = true)
 |-- Rate: string (nullable = true)
 |-- DiscountPercent: string (nullable = true)
 |-- PaymentTermsDays: string (nullable = true)



In [140]:
from pyspark.sql.functions import col, trim, upper
from pyspark.sql.types import IntegerType, DoubleType

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 142, Finished, Available, Finished, False)

In [141]:
df_silver = (
    df
    .withColumn("SalesID", trim(upper(col("SalesID"))))
    .withColumn("PlantID", trim(upper(col("PlantID"))))
    .withColumn("CustomerID", trim(upper(col("CustomerID"))))
    .withColumn("UnitsSold", col("UnitsSold").cast(IntegerType()))
    .withColumn("Rate", col("Rate").cast(DoubleType()))
    .withColumn("DiscountPercent", col("DiscountPercent").cast(DoubleType()))
    .withColumn("PaymentTermsDays", col("PaymentTermsDays").cast(IntegerType()))
    # .withColumn("Date", col("Date").cast(int ()))
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 143, Finished, Available, Finished, False)

In [142]:
from pyspark.sql.functions import to_date

df_silver = df_silver.withColumn(
    "Date",
    date_format(to_date(col("Date"), "dd-MM-yyyy"), "ddMMyyyy").cast("int")
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 144, Finished, Available, Finished, False)

In [143]:
display(df_silver)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 145, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 187f5753-0097-4d1e-89ec-e3f7e1fb03d7)

In [144]:
from pyspark.sql.functions import col

df_silver = df_silver.withColumn("Date", col("Date").cast("int"))


StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 146, Finished, Available, Finished, False)

In [145]:
df_silver.printSchema()

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 147, Finished, Available, Finished, False)

root
 |-- SalesID: string (nullable = true)
 |-- Date: integer (nullable = true)
 |-- PlantID: string (nullable = true)
 |-- CustomerID: string (nullable = true)
 |-- UnitsSold: integer (nullable = true)
 |-- Rate: double (nullable = true)
 |-- DiscountPercent: double (nullable = true)
 |-- PaymentTermsDays: integer (nullable = true)



In [146]:
display(df_silver)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 148, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 6d842d9e-5395-4c49-bd7d-a00c61e0d87f)

In [147]:
df_silver = df_silver.filter(col("UnitsSold") > 0)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 149, Finished, Available, Finished, False)

In [148]:
df_silver = df_silver.withColumn(
    "NetUnitPrice",
    col("Rate") * (1 - col("DiscountPercent") / 100)
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 150, Finished, Available, Finished, False)

In [149]:
df_silver = df_silver.withColumn(
    "InvoiceAmount",
    col("UnitsSold") * col("NetUnitPrice")
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 151, Finished, Available, Finished, False)

In [150]:
df_silver = df_silver.withColumn(
    "TaxAmount",
    col("InvoiceAmount") * 0.18
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 152, Finished, Available, Finished, False)

In [151]:
df_silver = df_silver.withColumn(
    "TotalInvoiceAmount",
    col("InvoiceAmount") + col("TaxAmount")
)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 153, Finished, Available, Finished, False)

In [152]:
from pyspark.sql.functions import col, to_date, format_string

df_silver = df_silver.withColumn(
    "InvoiceDate",
    to_date(col("Date").cast("string"), "ddMMyyyy")
)


StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 154, Finished, Available, Finished, False)

In [153]:
from pyspark.sql.functions import to_date, expr

df_silver = df_silver.withColumn(
    "InvoiceDate",
    to_date(col("Date"))
)

df_silver = df_silver.withColumn(
    "DueDate",
    expr("date_add(InvoiceDate, PaymentTermsDays)")
)


StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 155, Finished, Available, Finished, False)

In [154]:
from pyspark.sql.functions import when

df_silver = df_silver.withColumn(
    "HighDiscountFlag",
    when(col("DiscountPercent") > 30, 1).otherwise(0)
)


StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 156, Finished, Available, Finished, False)

In [135]:
silver_path = "abfss://Fabric_Test@onelake.dfs.fabric.microsoft.com/Test_Lakehouse.Lakehouse/Files/Silver Layer Files"
df_silver.write.mode("overwrite") \
    .parquet(f"{silver_path}/silver_Sales")

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 137, Finished, Available, Finished, False)

In [155]:
df_check = spark.read.parquet(f"{silver_path}/silver_Sales")
display(df_check)

StatementMeta(, 9c595c37-a387-41f4-bb79-e4d8cec27480, 157, Finished, Available, Finished, False)

SynapseWidget(Synapse.DataFrame, 256fdf09-8f2e-472c-b194-8ec3bcbc3eb2)